In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
%matplotlib inline

# Establishing connection & Creating Database

In [3]:
client = pymongo.MongoClient('localhost',27017)

In [4]:
db = client['Income-database']

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Income-database')

# Creating Collections

In [6]:
collection = db['Income-collections']

In [7]:
collection.count()

C:\Users\91956\AppData\Local\Temp/ipykernel_27316/2497048341.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count()


0

# Reading and Inserting data

In [8]:
with open('Income_Header.txt') as f_in:
    column_headers = f_in.readline()

In [9]:
column_headers

'age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label'

In [10]:
column_head_list = column_headers.split(',')

In [11]:
column_head_list

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'label']

In [17]:
row_dict_list = list()

In [19]:
# with open('Income.txt') as f_in:
#     for line in f_in:
#         row_list = line.rstrip('\n').split(',')
#         row_dict = dict(zip(column_head_list,row_list))
#         row_dict_list.append(row_dict)
#         collection.insert_one(row_dict)

In [12]:
# collection.count()

In [15]:
# collection.delete_many({})

# Data Pre-Processing

In [16]:
with open('Income.txt') as f_in:
    for line in f_in:
        row_list = line.rstrip('\n').split(',')
        row_dict = dict(zip(column_head_list,row_list))
        try:
            row_dict['age'] = int(row_dict['age'])
            row_dict['hours-per-week'] = int(row_dict['hours-per-week'])
            collection.insert_one(row_dict)
        except:
            pass

In [17]:
collection.count()

C:\Users\91956\AppData\Local\Temp/ipykernel_27316/2497048341.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count()


32561

In [18]:
over_35 = collection.find({'age':{'$gt':35}})

In [19]:
type(over_35)

pymongo.cursor.Cursor

In [20]:
over_35.next()

{'_id': ObjectId('65c29e353f585d18616c5104'),
 'age': 39,
 'workclass': ' State-gov',
 'fnlwgt': ' 77516',
 'education': ' Bachelors',
 'education-num': ' 13',
 'marital-status': ' Never-married',
 'occupation': ' Adm-clerical',
 'relationship': ' Not-in-family',
 'race': ' White',
 'sex': ' Male',
 'capital-gain': ' 2174',
 'capital-loss': ' 0',
 'hours-per-week': 40,
 'native-country': ' United-States',
 'label': ' <=50K'}

In [21]:
over_35.count()

C:\Users\91956\AppData\Local\Temp/ipykernel_27316/3686870910.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  over_35.count()


17636

In [34]:
# import time
# start = time.time()
# agegt50 = collection.find({'age': {'$gt': 50}})
# end = time.time()
# print(end - start)

In [22]:
# age50.count()

# Indexing - improve query performance

In [24]:
index_results = db.profile.create_index([('age',pymongo.ASCENDING)],unique=False)

In [23]:
# start = time.time()
# agegt30 = collection.find({'age': {'$gt': 30}})
# end = time.time()
# print(end - start)

In [25]:
income_df = pd.DataFrame(list(collection.find()))

In [27]:
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   _id             32561 non-null  object
 1   age             32561 non-null  int64 
 2   workclass       32561 non-null  object
 3   fnlwgt          32561 non-null  object
 4   education       32561 non-null  object
 5   education-num   32561 non-null  object
 6   marital-status  32561 non-null  object
 7   occupation      32561 non-null  object
 8   relationship    32561 non-null  object
 9   race            32561 non-null  object
 10  sex             32561 non-null  object
 11  capital-gain    32561 non-null  object
 12  capital-loss    32561 non-null  object
 13  hours-per-week  32561 non-null  int64 
 14  native-country  32561 non-null  object
 15  label           32561 non-null  object
dtypes: int64(2), object(14)
memory usage: 4.0+ MB


In [29]:
income_df.dtypes

_id               object
age                int64
workclass         object
fnlwgt            object
education         object
education-num     object
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain      object
capital-loss      object
hours-per-week     int64
native-country    object
label             object
dtype: object

In [31]:
income_df.head()

,_id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,65c29e353f585d18616c5104,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,65c29e353f585d18616c5105,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,65c29e353f585d18616c5106,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,65c29e353f585d18616c5107,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,65c29e353f585d18616c5108,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [30]:
income_df['age'].describe()

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [33]:
income_df['education'].value_counts()

 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int64